### Import Libaries

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.distributions as dist
from torchsummary import summary
import math
import os
import numpy as np
import time
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pathlib import Path
import re
from skimage.metrics  import structural_similarity as ssim
import plotly.io as pio
import plotly.express as px
import pandas as pd

pio.renderers.default = 'iframe'

from importlib import reload


# locals
import model_architectures
import visualization
import unet

reload(model_architectures)
from model_architectures import VAESegment, Data3DSingleSegT2, SegMaskData

reload(unet)
from unet import UNet

reload(visualization)
from visualization import brain_diff, viz_slices

C:\compute\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define Paths

In [4]:
research_dir = r"D:/school/research"
code_dir = os.path.join(research_dir, "code")
model_dir = os.path.join(code_dir, "explore_again", "models")
data_dir = os.path.join(research_dir, "data")
dhcp_rel2 = os.path.join(data_dir, "dhcp_rel2")
processed_dir = os.path.join(dhcp_rel2, "processed")
volume_dir = os.path.join(processed_dir, "volumes")
seg_dir = os.path.join(processed_dir, "segments")
seg_vol_dir = os.path.join(processed_dir, "volume_segments")
pred_dir = os.path.join(dhcp_rel2, "predictions")
seg_pred_dir = os.path.join(pred_dir, "vae_9seg")

l1_dir = os.path.join(volume_dir, "l1")
l5_dir = os.path.join(volume_dir, "l5")

l1_seg_dir = os.path.join(seg_dir, "l1")
l5_seg_dir = os.path.join(seg_dir, "l5")

l1_seg_vol_dir = os.path.join(seg_vol_dir, "l1")
l5_seg_vol_dir = os.path.join(seg_vol_dir, "l5")

l1_seg_pred_dir = os.path.join(seg_pred_dir, "l1")
l5_seg_pred_dir = os.path.join(seg_pred_dir, "l5")

metrics_dir = os.path.join(code_dir, "explore_again", "metrics", "individual_segments")

### Define Data Parameters

In [5]:
np.random.seed(42)
num_samples = int(len(os.listdir(l1_dir)) / 2)
samples = np.array([i for i in range(0, num_samples)])
np.random.shuffle(samples)

split_val = int(0.8 * num_samples)
train_indices = samples[0:split_val]
val_indices = samples[split_val:]

num_test = int(len(os.listdir(l5_dir)) / 2)
test_indices = np.array([i for i in range(0, num_test)])

#### Assess Model for Each Segment

In [6]:
segments = [
    "Cerebrospinal Fluid",
    "Cortical Grey Matter",
    "White Matter",
    "Background",
    "Ventricle",
    "Cerebelum",
    "Deep Grey Matter",
    "Brainstem",
    "Hippocampus"
]

In [25]:
def get_losses(data, batch, segment):
    seg_losses = []
    for idx, img in enumerate(data):
        x = torch.Tensor(np.reshape(img[0], (1, 1, 256, 256, 256))).cuda()
        output = model(x)
        pred = np.reshape(output.cpu().detach().numpy(), (1, 256, 256, 256))
        seg_losses.append(((img[1] - pred)**2).mean())
    
    loss_path = os.path.join(metrics_dir, f"vae_rel2t2_seg{segment}_{batch}_metrics.npy")
    np.save(loss_path, np.array(seg_losses))
    
    return seg_losses

In [26]:
%%time

for segment_number in range(0, len(segments)):
    print(f"Train model for segment {segment_number}")
    
    # Load data for segment
    train = Data3DSingleSegT2(l1_dir, l1_seg_vol_dir, train_indices, segment=segment_number)
    val = Data3DSingleSegT2(l1_dir, l1_seg_vol_dir, val_indices, segment=segment_number)
    test = Data3DSingleSegT2(l5_dir, l5_seg_vol_dir, test_indices, segment=segment_number)
    
    # Define output paths now :)
    model_path = os.path.join(model_dir, f"vae_rel2t2_seg{segment_number}.pt")
    model = VAESegment(1, 1)
    model.load_state_dict(torch.load(model_path))
    model.cuda()
    model.eval()
    
    seg_losses1 = get_losses(train, "train", segment_number)
    seg_losses2 = get_losses(val, "val", segment_number)
    seg_losses3 = get_losses(test, "test", segment_number)

Train model for segment 0
Train model for segment 1
Train model for segment 2
Train model for segment 3
Train model for segment 4
Train model for segment 5
Train model for segment 6
Train model for segment 7
Train model for segment 8
CPU times: total: 18min 43s
Wall time: 25min 31s


In [19]:
# Single segment models
segment_label = []
run_name = []
values = []

for segment_number in range(0, len(segments)):
    for batch in ["train", "val", "test"]:
        losses = np.load(os.path.join(metrics_dir, f"vae_rel2t2_seg{segment_number}_{batch}_metrics.npy"))
        segment_label.extend([f"{segments[segment_number]} (Single)" for x in range(0, len(losses))])
        run_name.extend([batch.capitalize() for x in range(0, len(losses))])
        values.extend(losses)

In [20]:
# All In One Model

aio_metrics_dir = os.path.join(code_dir, "explore_again", "metrics", "combined_segments")

train_seg_losses = np.load(os.path.join(aio_metrics_dir, "train_seg_losses.npy"))
val_seg_losses = np.load(os.path.join(aio_metrics_dir, "val_seg_losses.npy"))
test_seg_losses = np.load(os.path.join(aio_metrics_dir, "test_seg_losses.npy"))

def get_values(list_o_vals, run_name):
    s = []
    v = []
    r = []
    for m in list_o_vals:
        for idx, n in enumerate(m):
            s.append(f"{segments[idx]} (AIO)")
            v.append(n)
            r.append(run_name)
    return s, v, r

train_labels, train_values, train_runs = get_values(train_seg_losses, "Train")
val_labels, val_values, val_runs = get_values(val_seg_losses, "Val")
test_labels, test_values, test_runs = get_values(test_seg_losses, "Test")

values.extend(train_values)
run_name.extend(train_runs)
segment_label.extend(train_labels)

values.extend(val_values)
run_name.extend(val_runs)
segment_label.extend(val_labels)

values.extend(test_values)
run_name.extend(test_runs)
segment_label.extend(test_labels)

segment_df = pd.DataFrame({"run": run_name, "segment": segment_label, "values": values})

array(['Background (AIO)', 'Background (Single)', 'Brainstem (AIO)',
       'Brainstem (Single)', 'Cerebelum (AIO)', 'Cerebelum (Single)',
       'Cerebrospinal Fluid (AIO)', 'Cerebrospinal Fluid (Single)',
       'Cortical Grey Matter (AIO)', 'Cortical Grey Matter (Single)',
       'Deep Grey Matter (AIO)', 'Deep Grey Matter (Single)',
       'Hippocampus (AIO)', 'Hippocampus (Single)', 'Ventricle (AIO)',
       'Ventricle (Single)', 'White Matter (AIO)',
       'White Matter (Single)'], dtype=object)

In [26]:
segment_df.sort_values(by=['segment'])

fig = px.box(
    segment_df,
    x="segment",
    y="values",
    # facet_col= "marker",
    color="run",
    color_discrete_map={
        "Train": "green",
        "Val": "orange",
        "Test": "red"
    },
    category_orders={
        "segment": np.unique(segment_df["segment"])
    }
)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
# fig.show()